VERSAO JANEIRO 2018

In [88]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import *

Definicao de funcoes

In [89]:
#equacao de movimento para x2pontos
def Fx(m,x,y):
    return (- x - 2*x*y)/m

#equacao de movimento para y2pontos
def Fy(m,x,y):
    return (-x**2 -y + y**2)/m

#mesmas equacoes em coordenadas polares
def Fr(m,r,th): #r2pontos
    return (-r -3*(r**2)*np.sin(th)*(np.cos(th))**2 - (r**2)*np.sin(th)**3)/m

def Fth(m,r,th): #theta2pontos
    return (r**3)*(2*np.cos(th)*(np.sin(th))**2 - np.cos(th)**3 + np.cos(th)*np.sin(th)**2)/m

#Hamiltoniana e Potencial em Coordenadas Cartesianas
def H(vx,vy,x,y):
    return 0.5*(vx**2 + vy**2 + x**2 + y**2) + (x**2)*y -(y**3)/3

def V(x,y):
    return 0.5*(x**2 + y**2) + y*(x**2) - (y**3)/3

#Hamiltoniana e Potencial em Coordenadas Polares
def Hp(r,theta,vr,vth):
    #return 0.5*(vr**2 + r**2) + r*vr*(np.sin(theta)*np.sin(vth) + np.cos(theta)*np.cos(vth)) + (1/2)*r**2 + (np.sin(theta)*np.cos(theta)**2 - (1/3)*np.sin(theta)**3)*r**3
    return 0.5*(vr**2 + r**2 + vth*(r**2)) + (r**3)*np.sin(theta)*((np.cos(theta)**2)-(np.sin(theta)**2)/3)

def Vp(r,theta): #V=V(r,theta)
    return (1/2)*r**2 + (np.sin(theta)*(np.cos(theta)**2) - (1/3)*np.sin(theta)**3)*r**3

Construcao das curvas de superficies equipotenciais

In [90]:
#H para K = 0, formando surperficies equipotenciais
N = 100000 #quantidade de passos
h = 9/N #tamanho do passo (5/N)
yl = np.linspace(-1.5,1.5,N) #valores de y para as curvas
c = [i* .4/20 for i in range(1,20)] #conjunto de valores para energia potencial

#valores de x para as curvas
def xl(yl,c):
    return np.sqrt((c + (1/3)*yl**3 - (1/2)*yl**2)/(1/2 + yl))

#plot das curvas
fig = plt.figure()
for hl in c:
    if (hl <= 0.16):
        plt.plot(xl(yl,hl),yl,'r-')
        plt.plot(-xl(yl,hl),yl,'r-')
    else:
        plt.plot(xl(yl,hl),yl,'k-')
        plt.plot(-xl(yl,hl),yl,'k-')
plt.axis([-1.0,1.0,-1.0,1.0])
plt.show()

C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


Bloco da definicao dos termos para o Runge-Kutta na forma de listas de extensao N

In [91]:
#definicao de posicoes x, y e r e theta
x = list(range(N))
y = list(range(N))
r = list(range(N))
th = list(range(N))

#definicao de velocidades vx, vy e vr e vtheta
vx = list(range(N))
vy = list(range(N))
vr = list(range(N))
vth = list(range(N))

#definicao dos termos do Runge-Kutta de 4a ordem (em cartesianas)
k1vx = list(range(N))
k1vy = list(range(N))
k1x = list(range(N))
k1y = list(range(N))

k2vx = list(range(N))
k2vy = list(range(N))
k2x = list(range(N))
k2y = list(range(N))

k3vx = list(range(N))
k3vy = list(range(N))
k3x = list(range(N))
k3y = list(range(N))

k4vx = list(range(N))
k4vy = list(range(N))
k4x = list(range(N))
k4y = list(range(N))

Construção das condições iniciais

In [94]:
#-----------------------
#construcao das condicoes iniciais em r e theta, em seguida faz-se a transformacao para coordenadas cartesianas
#-----------------------

#rCond=np.linspace(0.001,1.13,5)#valores adequados ao estabelecimento das condições iniciais
#thCond=np.linspace(0,2*np.pi,5)
#xCond=[]
#yCond=[]
#rVal=[]
#thVal=[]

#for i in range(0,len(rCond)-1):
#    for j in range(0,len(thCond)-1):
#        if (Vp(rCond[i],thCond[j])<0.16):
#            rVal.append(rCond[i])
#            thVal.append(thCond[i])
#            xCond.append(rCond[i]*np.cos(thCond[j]))
#            yCond.append(rCond[i]*np.sin(thCond[j]))
#    if (rCond[i]==0):
#        continue
#print(len(rVal))
#print(len(xCond))

#-----------------------
#construcao das condicoes iniciais em x e y
#-----------------------

xVal=np.linspace(-0.8,0.8,10)
yVal=np.linspace(-0.4,0.8,10)
xCond=[]
yCond=[]

for i in range(0,len(xVal)-1):
    for j in range(0,len(yVal)-1):
        if (V(xVal[i],yVal[j])<0.16):
            xCond.append(xVal[i])
            yCond.append(yVal[j])

print(len(xCond))

plt.plot(xCond,yCond,'go')
plt.show()

48


Bloco do Runge-Kutta. Primeiro loop, condicoes iniciais; segundo loop, Runge-Kutta.

In [95]:
#Fazendo Runge kutta nas equações de movimento para r e theta

fig=plt.figure()

#vthVal=[]
vxCond=[]

#vr = 0 #já foi definido anteriormente no bloco das variaveis
vyCond = 0.0

for k in range(0,len(xCond)-1):#construcao do vetor vthVal com os valores iniciais de vth 
    #aux=(2*(0.17-(rVal[k]**3)*np.sin(thVal[k])*((np.cos(thVal[k])**2)-(np.sin(thVal[k])**2)/3))-rVal[k]**2-vr**2)/(rVal[k]**2)
    #vthVal.append(np.sqrt(aux))
    
    aux = 2*(0.3 - (xCond[k]**2)*yCond[k] + (yCond[k]**3)/3) - xCond[k]**2 - yCond[k]**2 - vyCond**2
    vxCond.append(np.sqrt(aux))
    
for j in range(0,1): #atribuicao de condicoes iniciais
    #-----------
    #com condicoes iniciais em polares
    #-----------
    
    #x[0] = xCond[j]
    #y[0] = yCond[j]
    #r[0] = rVal[j]
    #th[0] = thVal[j]
    #vth[0] = vthVal[j] #velocidade angular vth[0] = vthVal = np.sqrt(aux)
    #vx[0] = -r[0]*np.sin(th[0])*vth[0]+np.cos(th[0])*vr
    #vy[0] = r[0]*np.cos(th[0])*vth[0]+np.sin(th[0])*vr
    
    #-----------
    #com condicoes iniciais em cartesianas
    #-----------
    
    x[0] = xCond[j]
    y[0] = yCond[j]
    vx[0]=vxCond[j]
    vy[0]=vyCond
    
    for i in range(0,N-1): #execucao do metodo de Runge-Kutta com coordenadas cartesianas
        k1vx[i] = Fx(1,x[i],y[i])*h
        k1vy[i] = Fy(1,x[i],y[i])*h
        k1x[i] = vx[i]*h
        k1y[i] = vy[i]*h
    
        k2vx[i] = Fx(1,x[i]+k1x[i]/2,y[i]+k1y[i]/2)*h
        k2vy[i] = Fy(1,x[i]+k1x[i]/2,y[i]+k1y[i]/2)*h
        k2x[i] = (vx[i] + k1vx[i]/2)*h
        k2y[i] = (vy[i] + k1vy[i]/2)*h
    
        k3vx[i] = Fx(1,x[i]+k2x[i]/2,y[i]+k2y[i]/2)*h
        k3vy[i] = Fy(1,x[i]+k2x[i]/2,y[i]+k2y[i]/2)*h
        k3x[i] = (vx[i] + k2vx[i]/2)*h
        k3y[i] = (vy[i] + k2vy[i]/2)*h
    
        k4vx[i] = Fx(1,x[i]+k3x[i],y[i]+k3y[i])*h
        k4vy[i] = Fy(1,x[i]+k3x[i],y[i]+k3y[i])*h
        k4x[i] = (vx[i] + k3vx[i])*h
        k4y[i] = (vy[i] + k3vy[i])*h
    
        vx[i+1] = vx[i] + (1/6)*(k1vx[i] + 2*(k2vx[i] + k3vx[i]) + k4vx[i])
        vy[i+1] = vy[i] + (1/6)*(k1vy[i] + 2*(k2vy[i] + k3vy[i]) + k4vy[i])
    
        x[i+1] = x[i] + (1/6)*(k1x[i] + 2*(k2x[i] + k3x[i]) + k4x[i])
        y[i+1] = y[i] + (1/6)*(k1y[i] + 2*(k2y[i] + k3y[i]) + k4y[i])
        
        print(H(vx[i+1],vy[i+1],x[i+1],y[i+1]))
    
    if (x[i+1]>np.sqrt(3)/2): #(x[i+1]>1.5 and y[i+1]<-0.5):se sair pela direita
        plt.plot(x[0],y[0],'b.')#azul
    elif (x[i+1]<-np.sqrt(3)/2): #(x[i+1]<-1.5 and y[i+1]<-0.5):se sair pela esquerda
        plt.plot(x[0],y[0],'r.')#vermelho
    elif (y[i+1]>1.0): #(y[i+1]>2.0 and abs(x[i+1])<0.27):se sair por cima
        plt.plot(x[0],y[0],'g.')#verde
    else: #se nao sair
        plt.plot(x[0],y[0],'k.')#preto
            
plt.axis([-1.5,1.5,-1.5,1.5]) #extensao da janela do plot
plt.plot(xl(yl,0.16),yl,'r-') #plot de metade da curva equipotencial
plt.plot(-xl(yl,0.16),yl,'r-') #plot da outra metade da curva equipotencial
plt.show() #exibicao do plot

0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3


C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: overflow encountered in double_scalars
  app.launch_new_instance()
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in sqrt
